In [1]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
import torchvision
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim

##### Custom Dataloader definition

In [2]:
class WoundDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index])
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0
        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]
        return image, mask

#### UNET Model Definition

In [3]:
# how to add dropout in UNET architecture
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

#### Utility Function Definitions

In [4]:
def save_checkpoint(state, filename="my_checkpoint_large_04lr.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    train_ds = WoundDataset(
        image_dir=train_dir,
        mask_dir=train_maskdir,
        transform=train_transform,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = WoundDataset(
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader

def save_predictions_as_imgs(
    loader, model, folder="prediction_images/", device="cuda"
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(
            preds, f"{folder}/pred_{idx}.png"
        )
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}{idx}.png")

    model.train()

#### Train & Validation Function Definitions

In [5]:
def validate_model(loader, model, device="cuda"):
    """Test the network on the training set."""
    print("~~~~ In test ~~~~")
    criterion = torch.nn.BCEWithLogitsLoss()
    loss = 0
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    iou_score  = 0
    result = []
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            loss += criterion(preds, y).item()
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds.sum() + y.sum()) + 1e-8
            )
            # Calculate IoU score
            intersection = (preds * y).sum()
            union = (preds + y).sum() - intersection
            iou_score += (intersection + 1e-8) / (union + 1e-8)

    num_batches = len(loader)
    loss /= num_batches
    acc = num_correct/num_pixels*100
    diceS = dice_score/num_batches
    iouS = iou_score/num_batches
    correct_pixel = num_correct
    total_pixel = num_pixels
    print(f"Loss = {loss}")
    print(f"IoU Score = {iouS}")
    print(f"Dice Score = {diceS}")
    print("~~~~~ Out of test ~~~~~")

    model.train()
    
    result = [acc.item(), diceS.item(), iouS.item(), loss, correct_pixel.item(), total_pixel]
    
    return result

def train_model(loader, model, optimizer, loss_fn, scaler):
    print("In Train")
    print(f'length of loader {len(loader)}')
    model.train()
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())
    print("Out of Train")

#### Define Parameters

In [6]:
# Hyperparameters etc.
LEARNING_RATE = 0.00001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 200
NUM_WORKERS = 0
IMAGE_HEIGHT = 240 
IMAGE_WIDTH = 240  
PIN_MEMORY = True
LOAD_MODEL = False
# TRAIN_IMG_DIR = "../data/train_data_small/train_images/"
# TRAIN_MASK_DIR = "../data/train_data_small/train_masks/"
# VAL_IMG_DIR = "../data/validation_data/val_images/"
# VAL_MASK_DIR = "../data/validation_data/val_masks/"
TRAIN_IMG_DIR = "../data/woundData/train_images/"
TRAIN_MASK_DIR = "../data/woundData/train_masks/"
VAL_IMG_DIR = "../data/woundData/val_images/"
VAL_MASK_DIR = "../data/woundData/val_masks/"
csv_filename = "output_e" + str(NUM_EPOCHS) + "bs" + str(BATCH_SIZE) + "lr" + str(LEARNING_RATE) + ".csv"

In [7]:
print(DEVICE)

cuda


#### Image transform function

In [8]:
# used to transform training set
train_transform = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

# used to transform validation set
val_transforms = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)


In [9]:
column_names = {'epoch':[]
                , 'accuracy':[]
                , 'dice_score':[]
                , 'IoU_score':[]
                , 'Loss': []
                , 'correct_pixels':[]
                , 'total_pixel':[]}
df = pd.DataFrame(columns=column_names)


labels = ["Accuracy", "Dice Score", "IOU", "Loss", "Correct Pixel", "Total Pixel"]

train_loader, val_loader = get_loaders(
    TRAIN_IMG_DIR,
    TRAIN_MASK_DIR,
    VAL_IMG_DIR,
    VAL_MASK_DIR,
    BATCH_SIZE,
    train_transform,
    val_transforms,
    NUM_WORKERS,
    PIN_MEMORY,
)


model = UNET().to(DEVICE)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

if LOAD_MODEL:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)

single_epoch_result = validate_model(val_loader, model, device=DEVICE)
print(f'length of train loader -> {len(train_loader)}')
print("First Check - Before training starts")
for label, item in zip(labels, single_epoch_result):
    print(f"{label}: {item}")
    
scaler = torch.cuda.amp.GradScaler()

for epoch in range(NUM_EPOCHS):
    single_epoch_result = []
    train_model(train_loader, model, optimizer, loss_fn, scaler)
    
    # save model
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer":optimizer.state_dict(),
    }
    save_checkpoint(checkpoint)

    # Saving model accuracy to a csv 
    single_epoch_result = validate_model(val_loader, model)
    
    print(f'~~~~~epoch => {epoch}')
    for label, item in zip(labels, single_epoch_result):
        print(f"{label}: {item}")
    print('~~~~~~~~~~~~~~~~~end epoch')
    
    single_epoch_result.insert(0, epoch)
    df.loc[len(df)] = single_epoch_result
    #df = df.concat(single_epoch_result)

    # print some examples to a folderthu
    save_predictions_as_imgs(
        val_loader, model, folder="prediction_images/", device=DEVICE
    )


# Save the DataFrame as a CSV file
csv_path = 'dataframes/' + csv_filename
df.to_csv(csv_path, index=False)


~~~~ In test ~~~~
Loss = 0.985631539271428
IoU Score = 0.013426982797682285
Dice Score = 0.02647390589118004
~~~~~ Out of test ~~~~~
length of train loader -> 39
First Check - Before training starts
Accuracy: 1.353515625
Dice Score: 0.02647390589118004
IOU: 0.013426982797682285
Loss: 0.985631539271428
Correct Pixel: 155925
Total Pixel: 11520000
In Train
length of loader 39


100%|██████████████████████████████████████████████████████████████████████| 39/39 [00:16<00:00,  2.41it/s, loss=0.683]


Out of Train
=> Saving checkpoint
~~~~ In test ~~~~
Loss = 0.9570801716584426
IoU Score = 0.018705179914832115
Dice Score = 0.036642301827669144
~~~~~ Out of test ~~~~~
~~~~~epoch => 0
Accuracy: 68.32682800292969
Dice Score: 0.036642301827669144
IOU: 0.018705179914832115
Loss: 0.9570801716584426
Correct Pixel: 7871250
Total Pixel: 11520000
~~~~~~~~~~~~~~~~~end epoch
In Train
length of loader 39


100%|██████████████████████████████████████████████████████████████████████| 39/39 [00:15<00:00,  2.48it/s, loss=0.635]


Out of Train
=> Saving checkpoint
~~~~ In test ~~~~
Loss = 0.9257071568415716
IoU Score = 0.0933094248175621
Dice Score = 0.16935494542121887
~~~~~ Out of test ~~~~~
~~~~~epoch => 1
Accuracy: 90.68452453613281
Dice Score: 0.16935494542121887
IOU: 0.0933094248175621
Loss: 0.9257071568415716
Correct Pixel: 10446857
Total Pixel: 11520000
~~~~~~~~~~~~~~~~~end epoch
In Train
length of loader 39


...

Out of Train
=> Saving checkpoint
~~~~ In test ~~~~
Loss = 0.7321170293367826
IoU Score = 0.6437798738479614
Dice Score = 0.7754225134849548
~~~~~ Out of test ~~~~~
~~~~~epoch => 197
Accuracy: 99.47032928466797
Dice Score: 0.7754225134849548
IOU: 0.6437798738479614
Loss: 0.7321170293367826
Correct Pixel: 11458982
Total Pixel: 11520000
~~~~~~~~~~~~~~~~~end epoch
In Train
length of loader 39


100%|█████████████████████████████████████████████████████████████████████| 39/39 [00:15<00:00,  2.54it/s, loss=0.0914]


Out of Train
=> Saving checkpoint
~~~~ In test ~~~~
Loss = 0.7319488754639258
IoU Score = 0.6213933825492859
Dice Score = 0.7586614489555359
~~~~~ Out of test ~~~~~
~~~~~epoch => 198
Accuracy: 99.44461822509766
Dice Score: 0.7586614489555359
IOU: 0.6213933825492859
Loss: 0.7319488754639258
Correct Pixel: 11456020
Total Pixel: 11520000
~~~~~~~~~~~~~~~~~end epoch
In Train
length of loader 39


100%|██████████████████████████████████████████████████████████████████████| 39/39 [00:15<00:00,  2.54it/s, loss=0.107]


Out of Train
=> Saving checkpoint
~~~~ In test ~~~~
Loss = 0.7322520292722262
IoU Score = 0.6705346703529358
Dice Score = 0.7981967329978943
~~~~~ Out of test ~~~~~
~~~~~epoch => 199
Accuracy: 99.49109649658203
Dice Score: 0.7981967329978943
IOU: 0.6705346703529358
Loss: 0.7322520292722262
Correct Pixel: 11461374
Total Pixel: 11520000
~~~~~~~~~~~~~~~~~end epoch
